# OLSZTYN
- scraping details 
- DATE 2024 jan 16th - this is done using URLs downloaded on the 16th of January

In [1]:
import os
import subprocess

# prepering directories
path = os.getcwd()
# subfolders
input_dir = os.path.join(path, 'inputs')
output_dir = os.path.join(path, 'outputs')

# open folder
subprocess.Popen(f'explorer "{output_dir}"')

In [2]:
# selenium 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# BeautifulSoup
from bs4 import BeautifulSoup

import datetime
import time

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()


Bad key "text.kerning_factor" on line 4 in
C:\Users\Ukasz\Anaconda3\envs\geopandas\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
data = 'olsztyn_page_all_pages_20240116170027.csv'
date = datetime.datetime.now().strftime("%Y%m%d") # current date
new_file_name = f'{date}_olsztyn_details_test.csv'
df = pd.read_csv(os.path.join(os.getcwd(), 'outputs', data), index_col=False)
df = df.head(40)

In [4]:
# chrome driver location
PATH = os.path.join(path, 'inputs','chromedriver.exe' )

# chrom driver configuration
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
#options.add_argument('--headless') # without opening browser

driver = webdriver.Chrome(PATH, options=options)

wait = WebDriverWait(driver, 10)

In [5]:
# url 
url = 'https://www.otodom.pl'

In [6]:
'''
this piece of code adds 
uses a while loop and writes the data not found to expired or not_scraped sets
'''

not_scraped = set()
expired_set = set()
to_do = set(list(df.index))

top_tags = ['Powierzchnia','Forma własności','Liczba pokoi','Stan wykończenia','Piętro','Balkon / ogród / taras','Czynsz','Miejsce parkingowe','Obsługa zdalna','Ogrzewanie']
bottom_tags = ['Rynek', 'Typ ogłoszeniodawcy', 'Dostępne od','Rok budowy','Rodzaj zabudowy','Okna','Winda','Media','Zabezpieczenia','Wyposażenie','Informacje dodatkowe','Materiał budynku']

def accept_terms():
    # accepting terms and conditions
    # this has to be done only once
    element = wait.until(EC.element_to_be_clickable((By.ID, 'onetrust-pc-btn-handler')))
    element.click()
    # step two -  confirming preferences and closing the modal window
    element = wait.until(EC.element_to_be_clickable((By.XPATH, "//*[contains(text(), 'Potwierdzenie moich wyborów')]")))
    element.click()
    
    
def get_data(soup):
    data = {}
    top_information = soup.find('div', attrs={'data-testid':'ad.top-information.table'})
    bottom_information = soup.find('div', attrs={'data-testid':'ad.additional-information.table'})
       
    for tag in top_tags:
        try:
            div_tag = top_information.find('div', attrs={'aria-label':tag})
            results = div_tag.find_all("div")
            data[tag] = results[2].text
        except:
            #data[tag] = results[2].text
            data[tag] = 'brak informacji'
            
    for tag in bottom_tags:
        try:
            div_tag = bottom_information.find('div', attrs={'aria-label':tag})
            results = div_tag.find_all("div")
            data[tag] = results[2].text
        except:
            #data[tag] = results[2].text
            data[tag] = 'brak informacji'
    
    return data


def save_data(i, data, out_df, date):
    
    for k in data:
        out_df.loc[i, k] = data[k]
    out_df.to_csv(os.path.join(path, 'outputs', new_file_name), encoding='utf-8', index=False)

    
expired = 0
while list(not_scraped.union(to_do)):
    for i in list(df.index):
        date = datetime.datetime.now().strftime("%Y%m%d") # current date
        link = df['add_link'][i]
        # open website using webdriver
        driver.get(url + link)
        if i == 0:
            out_df = df.copy()
            out_df.to_csv(os.path.join(path, 'outputs', new_file_name), encoding='utf-8', index=False)
            accept_terms()

        time.sleep(2) # waits n seconds  
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # expired add
        if soup.findAll(text="To ogłoszenie jest już niedostępne"):
            expired += 1
            print(f'{i}_Expired', end='...')
            expired_set.add(i)
            to_do.remove(i)
            continue
        
        try:
            # data available in soup,  all as it should be
            data = get_data(soup)
            if i > 0:
                out_df = pd.read_csv(os.path.join(os.getcwd(), 'outputs', new_file_name), encoding='utf-8', index_col=False)   
            save_data(i, data, out_df, date)
            # remove id from to_do or from not_scraped
            if i in not_scraped:
                not_scraped.remove(i)
            if i in to_do:
                to_do.remove(i)
            print(i, end='...')
        except:
            # was not able to scrap data so id is added to not_scraped
            not_scraped.add(i)
            print(f'{i} Issue', end='...')

print('DONE !! ')

0...1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...DONE !! 
